# Custom Raster calculations using Numba
## ❓ Questions
- How do I create fast custom raster calculations that have more flexibility than basic operations?


## ❗ Objectives
- Create a numba compiled function that operates on multiple single pixels
- Create a numba compiled function that operates on multiple pixels at once


# What is Numba?
Python has several features that make it a great prototyping language, but it's native speed is often criticised. While this is ultimately down to the implementation, it is still an issue.  

Numpy and several other libraries will wrap compiled C, C++, or Fortran code to try to have the benefits of python without giving up too much speed.  

[Numba](https://numba.pydata.org/) takes a different approach:  
When you wrap a python function with a numba compilation 'decorator', it will try to compile it using the state-of-the-art LLVM compiler. One drawback is that it will only run with a subset of python and numpy functionality - most libraries will not be able to be compiled. One of the better features of numba is that you can write loops and the compiler will automatically optimise your code to be as fast or faster than single line numpy functions.  

"
You don't need to replace the Python interpreter, run a separate compilation step, or even have a C/C++ compiler installed. Just apply one of the Numba decorators to your Python function, and Numba does the rest. 
"

# Example function
Let's write an example function in numba to get the hang of it

In [28]:
from numba import njit
import numpy as np




In [30]:
# Timing the code


Note that while numba can create fast custom calculations, your implementation may not always be as fast as an existing implementation due to a number of factors

# Applying numba to remote sensing data
Let's load back in our data and create a function to apply to it

In [31]:
# Get the directory again
import os
from os.path import join

product_dir_textfile = "product_dir.txt"

with open(product_dir_textfile, 'r') as f:
    base_product_dir = f.readline()

product_dir = join(base_product_dir, 'GRANULE')
L2_dirname = os.listdir(product_dir)[0]
product_dir = join(product_dir, L2_dirname)
product_dir = join(product_dir, 'IMG_DATA', 'R60m')

# Get a dictionary of filenames
image_paths = {}
for fname in os.listdir(product_dir):
    fpath = join(product_dir, fname)
    file_band = fname.split('_')[2]
    image_paths[file_band] = fpath

image_paths

{'B12': '/data/ASDAF_Workshop_Materials_Remote_Sensing_Intro/notebooks/workshop_data/S2A_MSIL2A_20230824T021351_N0509_R060_T50HMK_20230824T070459/S2A_MSIL2A_20230824T021351_N0509_R060_T50HMK_20230824T070459.SAFE/GRANULE/L2A_T50HMK_A042671_20230824T022008/IMG_DATA/R60m/T50HMK_20230824T021351_B12_60m.jp2',
 'TCI': '/data/ASDAF_Workshop_Materials_Remote_Sensing_Intro/notebooks/workshop_data/S2A_MSIL2A_20230824T021351_N0509_R060_T50HMK_20230824T070459/S2A_MSIL2A_20230824T021351_N0509_R060_T50HMK_20230824T070459.SAFE/GRANULE/L2A_T50HMK_A042671_20230824T022008/IMG_DATA/R60m/T50HMK_20230824T021351_TCI_60m.jp2',
 'WVP': '/data/ASDAF_Workshop_Materials_Remote_Sensing_Intro/notebooks/workshop_data/S2A_MSIL2A_20230824T021351_N0509_R060_T50HMK_20230824T070459/S2A_MSIL2A_20230824T021351_N0509_R060_T50HMK_20230824T070459.SAFE/GRANULE/L2A_T50HMK_A042671_20230824T022008/IMG_DATA/R60m/T50HMK_20230824T021351_WVP_60m.jp2',
 'B06': '/data/ASDAF_Workshop_Materials_Remote_Sensing_Intro/notebooks/workshop_da

In [50]:
import rioxarray

red_int = rioxarray.open_rasterio(image_paths['B04'])
nir_int = rioxarray.open_rasterio(image_paths['B8A'])

In [51]:
# What data type is it?
red_int.dtype

dtype('uint16')

Let's convert the unint16 data to float32. To do this we will need to take the integer and divide by 10000 to get a reflectance

In [53]:
red = red_int.astype(np.float32)/10000
nir = ndvi_int.astype(np.float32)/10000
red.dtype

dtype('float32')

## NDVI
For some more practice, let's reimplement the NDVI using numba.


Time this code and compare it to the previous way of creating an NDVI. Is it worth it?

# Apply a kernel
We just applied a single pixel operation. Let's now use a more real-world use case: Applying a kernel over an image